In [1]:
#%matplotlib inline               
import pandas as pd              
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
np.random.seed(1337)  # for reproducibility
#import matplotlib.pyplot as plt

import scipy
import scipy.io.wavfile
import IPython
from sklearn.cluster import KMeans
import os 
import glob
#from python_speech_features import delta
#from python_speech_features import logfbank
import scipy.io.wavfile as wav
from sklearn import metrics
from sklearn.preprocessing import StandardScaler


import random
#from keras.datasets import mnist
#from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers
from keras import  regularizers
from keras.layers import LSTM
from keras.layers.normalization import BatchNormalization

from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
#batch_size = 25
#nb_classes = 14
#nb_epoch = 15

# input image dimensions
#img_rows, img_cols = 40, 25
# number of convolutional filters to use
#nb_filters = 32
# size of pooling area for max pooling
#pool_size = (3, 3)
# convolution kernel size
#kernel_size = (5, 3)

In [136]:
train_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_10sec_ivectors_X_train_04Jan_labels_ids_combined_extraction.csv')
#train_afds

In [137]:
#train_lre = train_lre.iloc[1000:2000]

In [138]:
train_lre.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
0,-0.314181,0.671920,-0.188068,-0.910752,1.050361,1.966829,0.292740,1.084547,1.287480,1.745248,...,-0.564911,-1.854116,-3.185948,-2.177057,0.301166,zkllk,LDC2017E22,data,eng-usg,zkllk
1,0.503734,0.247072,0.451463,0.500546,-0.716668,1.337575,-1.027019,0.773720,0.910362,1.728001,...,0.886613,6.766799,-1.040380,-0.751316,-1.487938,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562
2,2.141677,-0.044947,-0.595182,-0.595953,0.889385,1.440080,-1.208933,0.129059,-0.930121,0.401362,...,-2.836253,-0.323334,0.055760,0.481039,0.395826,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
3,1.708722,-0.663542,0.031733,0.358099,0.594610,1.023025,-1.450633,0.252159,-0.381832,0.087693,...,1.384728,0.426863,0.482178,1.655650,-1.029032,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
4,1.873292,-0.272493,-0.305948,-0.250047,2.093700,1.741634,-0.750633,-0.651771,-0.793287,-0.053527,...,0.188493,-0.888710,1.792536,-0.628455,1.912622,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a


In [139]:
train_lre.shape

(114276, 505)

In [140]:
#train_lre.groupby(['lang']).size()

In [141]:
#train_lre=train_lre.drop(train_lre.columns[0], axis=1)


In [142]:
#train_lre

In [143]:
val_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/dev_feat_BNF_h5_03Jan.csv')
#val_afds

In [144]:
#val_lre = val_lre.iloc[100:300]

In [145]:
val_lre.shape

(3661, 505)

In [146]:
val_lre.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,uttid,language_code,data_source,speech_duration,segmentid1
0,-0.888892,-0.725549,1.228262,2.474089,-0.136339,1.062172,-0.099022,-0.470705,1.654063,-1.702971,...,0.208072,0.467385,-0.336328,1.096394,-1.589369,lre17_jxjmxcks,spa-lac,mls14,30,lre17_jxjmxcks
1,-0.387867,-0.468537,-1.576730,0.049268,-0.716641,0.822740,0.408831,-1.029213,2.657424,0.756418,...,-1.132833,-0.464111,0.401425,-0.841559,1.031757,lre17_iaxztiph,zho-cmn,vast,1000,lre17_iaxztiph
2,-0.167892,-0.524584,-1.522211,-0.576471,1.031359,0.791165,2.272485,-1.107507,2.928776,2.028407,...,-1.956377,0.023279,-0.005259,1.020969,-0.648280,lre17_gzuiblur,por-brz,vast,1000,lre17_gzuiblur
3,-0.195398,-0.390386,-0.624269,1.535433,0.388117,1.727448,0.463109,2.059767,0.165206,0.619942,...,0.616712,-2.654504,0.482690,0.401316,-1.639292,lre17_zpuysjrb,zho-nan,mls14,10,lre17_zpuysjrb
4,0.192935,-1.009776,0.330685,2.114597,-1.960723,1.176015,0.320780,-0.202099,0.561382,-0.318042,...,0.680654,-1.863545,-2.006810,1.963633,-0.745066,lre17_amnlzmbw,qsl-pol,mls14,3,lre17_amnlzmbw


In [147]:
eval_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/eval_feat_BNF_h5_03Jan.csv')


In [148]:
eval_lre.shape

(25451, 505)

In [149]:
train_lre.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
0,-0.314181,0.671920,-0.188068,-0.910752,1.050361,1.966829,0.292740,1.084547,1.287480,1.745248,...,-0.564911,-1.854116,-3.185948,-2.177057,0.301166,zkllk,LDC2017E22,data,eng-usg,zkllk
1,0.503734,0.247072,0.451463,0.500546,-0.716668,1.337575,-1.027019,0.773720,0.910362,1.728001,...,0.886613,6.766799,-1.040380,-0.751316,-1.487938,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562
2,2.141677,-0.044947,-0.595182,-0.595953,0.889385,1.440080,-1.208933,0.129059,-0.930121,0.401362,...,-2.836253,-0.323334,0.055760,0.481039,0.395826,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
3,1.708722,-0.663542,0.031733,0.358099,0.594610,1.023025,-1.450633,0.252159,-0.381832,0.087693,...,1.384728,0.426863,0.482178,1.655650,-1.029032,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
4,1.873292,-0.272493,-0.305948,-0.250047,2.093700,1.741634,-0.750633,-0.651771,-0.793287,-0.053527,...,0.188493,-0.888710,1.792536,-0.628455,1.912622,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a


In [150]:
X_train = train_lre.drop(["lang",'ids','year','data','lang_id'],axis=1)
y_train = train_lre["lang"]

#y_train_uttid = train_lre["uttid"]

In [151]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,-0.314181,0.671920,-0.188068,-0.910752,1.050361,1.966829,0.292740,1.084547,1.287480,1.745248,...,-1.480749,0.210435,1.639710,-1.208895,-3.649134,-0.564911,-1.854116,-3.185948,-2.177057,0.301166
1,0.503734,0.247072,0.451463,0.500546,-0.716668,1.337575,-1.027019,0.773720,0.910362,1.728001,...,0.357329,0.321357,0.721421,2.759504,3.166255,0.886613,6.766799,-1.040380,-0.751316,-1.487938
2,2.141677,-0.044947,-0.595182,-0.595953,0.889385,1.440080,-1.208933,0.129059,-0.930121,0.401362,...,-0.396143,0.137315,-0.265354,-1.376924,0.890632,-2.836253,-0.323334,0.055760,0.481039,0.395826
3,1.708722,-0.663542,0.031733,0.358099,0.594610,1.023025,-1.450633,0.252159,-0.381832,0.087693,...,-0.188098,1.649831,0.232080,1.795104,0.627494,1.384728,0.426863,0.482178,1.655650,-1.029032
4,1.873292,-0.272493,-0.305948,-0.250047,2.093700,1.741634,-0.750633,-0.651771,-0.793287,-0.053527,...,-0.937242,0.223924,1.264577,-2.789978,1.040990,0.188493,-0.888710,1.792536,-0.628455,1.912622


In [152]:
y_train.head()

0    eng-usg
1    eng-usg
2    ara-apc
3    ara-apc
4    ara-apc
Name: lang, dtype: object

In [153]:
X_val = val_lre.drop(["language_code","uttid","segmentid1","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid1"]

In [154]:
type(X_val)

pandas.core.frame.DataFrame

In [155]:
X_eval = eval_lre.drop(["language_code","uttid","segmentid1","data_source","speech_duration"],axis=1)
y_eval = eval_lre["language_code"]
y_eval_segmentid = eval_lre["segmentid1"]

In [156]:
X_eval.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,-0.931934,0.576778,-1.876823,0.982179,0.335653,1.844014,0.945938,1.781178,0.290029,-0.906129,...,-0.099563,0.802841,-0.467284,0.483766,-0.888034,-1.816183,-0.753657,-0.468328,0.679314,0.577994
1,-0.476077,-0.957114,-0.533588,-0.301970,0.324679,1.065132,1.362624,-1.110453,-0.225287,0.862083,...,1.731532,0.420405,-0.126222,2.100710,0.642745,0.235046,-0.858469,-1.050246,0.941115,-2.466811
2,0.926436,0.644673,-0.139104,0.271413,-2.652249,1.879107,0.954021,-2.140020,1.109703,-1.294873,...,1.142919,0.802560,1.419063,-0.507672,1.469415,-0.286751,2.438928,1.930319,0.353751,-0.272758
3,-0.065790,-1.246687,-0.461387,-0.106515,0.411530,1.805592,2.254698,0.838559,1.319973,0.759386,...,0.258501,1.530374,0.527583,-2.245961,1.009446,1.811150,0.731798,-0.616567,2.438593,1.472902
4,0.470505,-1.737314,-0.393911,0.643027,-0.636450,1.067367,-0.551828,1.828177,1.220971,1.651914,...,-0.654729,0.415973,0.042833,0.312598,0.127827,-0.053129,0.817200,-0.513672,-0.370993,0.558443


In [157]:
y_eval.head()

0    zho-cmn
1    ara-ary
2    eng-usg
3    ara-arz
4    ara-arz
Name: language_code, dtype: object

In [158]:
#XX_train = X_train.append(X_val, ignore_index=True)

In [159]:
#type(XX_train)

In [160]:
#XX_train.shape

In [161]:
y_val.head()

0    spa-lac
1    zho-cmn
2    por-brz
3    zho-nan
4    qsl-pol
Name: language_code, dtype: object

In [162]:
y_train.head()

0    eng-usg
1    eng-usg
2    ara-apc
3    ara-apc
4    ara-apc
Name: lang, dtype: object

In [163]:
type(y_val)

pandas.core.series.Series

In [164]:
type(y_train)

pandas.core.series.Series

In [165]:
type(y_eval)

pandas.core.series.Series

In [166]:
#y_train = y_train.append(y_val, ignore_index=True)

In [167]:
#y_train.shape

In [168]:
y_train.head()

0    eng-usg
1    eng-usg
2    ara-apc
3    ara-apc
4    ara-apc
Name: lang, dtype: object

In [169]:
#X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [170]:
y_val_segmentid.head()

0    lre17_jxjmxcks
1    lre17_iaxztiph
2    lre17_gzuiblur
3    lre17_zpuysjrb
4    lre17_amnlzmbw
Name: segmentid1, dtype: object

In [171]:
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [172]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [173]:
y_train=le.transform(y_train)

In [174]:
y_train.shape

(114276,)

In [175]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

LabelEncoder()

In [176]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [177]:
y_val_labels = le.transform(y_val)

In [178]:
#y_val_labels[0:1000]

In [179]:
le = preprocessing.LabelEncoder()
le.fit(y_eval)

LabelEncoder()

In [180]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [181]:
y_eval_labels = le.transform(y_eval)

In [182]:
y_eval_labels[0:10]

array([12,  2,  5,  3,  3,  1, 12, 13,  6,  5])

In [183]:
X_train=X_train.values
X_val=X_val.values
X_eval=X_eval.values

In [184]:
X_train.shape

(114276, 500)

In [185]:
X_val.shape

(3661, 500)

In [186]:
X_eval.shape

(25451, 500)

In [187]:
nb_classes = 14

In [188]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
#Y_test = np_utils.to_categorical(y_test, nb_classes)
Y_val = np_utils.to_categorical(y_val_labels, nb_classes)
Y_eval = np_utils.to_categorical(y_eval_labels, nb_classes)

In [189]:
Y_eval

array([[ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [190]:
X_train.shape

(114276, 500)

In [191]:
X_val.shape

(3661, 500)

In [192]:
X_eval.shape

(25451, 500)

In [193]:
X_train = X_train.astype('float32')
#X_test = X_test.astype('float32')
X_val = X_val.astype('float32')
X_eval = X_eval.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
#print(X_test.shape[0], 'test samples')
print(X_val.shape[0], 'val samples')
print(X_eval.shape[0], 'eval samples')


X_train shape: (114276, 500)
114276 train samples
3661 val samples
25451 eval samples


In [194]:
model = Sequential()
model.add(Dense(512, input_dim=500))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
#model.add(BatchNormalization())
model.add(Activation('softmax'))

In [195]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 512)               256512    
_________________________________________________________________
activation_7 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_8 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 14)                7182      
__________

In [196]:
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [197]:
best_weights_filepath = '/home/satishk/best_weights_2l_MLP_combined.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [198]:
batch_size = 256
nb_epoch=20

In [199]:
history = model.fit(X_train, Y_train,batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_val , Y_val),callbacks=[saveBestModel])

Train on 114276 samples, validate on 3661 samples
Epoch 1/20
Epoch 00000: val_acc improved from -inf to 0.62824, saving model to /home/satishk/best_weights_2l_MLP_combined.hdf5
10s - loss: 0.8922 - acc: 0.7132 - val_loss: 1.2630 - val_acc: 0.6282
Epoch 2/20
Epoch 00001: val_acc improved from 0.62824 to 0.66758, saving model to /home/satishk/best_weights_2l_MLP_combined.hdf5
9s - loss: 0.1606 - acc: 0.9446 - val_loss: 1.1411 - val_acc: 0.6676
Epoch 3/20
Epoch 00002: val_acc improved from 0.66758 to 0.70036, saving model to /home/satishk/best_weights_2l_MLP_combined.hdf5
9s - loss: 0.1108 - acc: 0.9626 - val_loss: 1.1087 - val_acc: 0.7004
Epoch 4/20
Epoch 00003: val_acc improved from 0.70036 to 0.71292, saving model to /home/satishk/best_weights_2l_MLP_combined.hdf5
9s - loss: 0.0888 - acc: 0.9695 - val_loss: 1.1007 - val_acc: 0.7129
Epoch 5/20
Epoch 00004: val_acc did not improve
9s - loss: 0.0752 - acc: 0.9741 - val_loss: 1.2209 - val_acc: 0.7023
Epoch 6/20
Epoch 00005: val_acc did not

In [200]:
#reload best weights
model.load_weights(best_weights_filepath)
#model.load_weights('/home/q/Desktop/e10_512_32.hdf5')

In [201]:
#Frame label accuracy
score = model.evaluate(X_eval, Y_eval, verbose=0)
print('acc:', score[1])

acc: 0.716553377083


In [ ]:
train_lre.groupby(['langid']).size()

In [ ]:
train_lre.groupby(['langid']).size()

In [ ]:
train_lre.shape[0]

In [ ]:
aa=train_lre.groupby(['langid']).size()

In [ ]:
aa

In [ ]:
bb=aa.values

In [ ]:
bb

In [ ]:
lang_priors = bb/16205.0

In [ ]:
lang_priors

In [ ]:
type(lang_priors)

In [ ]:
lang_priors.shape

In [ ]:
log_lang_priors = np.log(lang_priors)

In [ ]:
posteriors = model.predict_proba(X_val)

In [ ]:
posteriors

In [ ]:
posteriors.shape

In [ ]:
log_posteriors = np.log(posteriors)

In [ ]:
log_likelihood = log_posteriors-log_lang_priors

In [ ]:
#log_likelihood[0]

In [ ]:
#type(log_likelihood)

In [ ]:
df_post = pd.DataFrame(log_likelihood)

In [ ]:
df_post

In [ ]:
df_post.columns = ['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg', 'por-brz', 'qsl-pol',
                   'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac', 'zho-cmn','zho-nan']

In [ ]:
df_post

In [ ]:
#pred_labels = model.predict_classes(X_val)

In [ ]:
#df_labels = pd.DataFrame(pred_labels)

In [ ]:
#df_labels

In [ ]:
#y_val_uttid

In [ ]:
df_y_val_segmentid=  pd.DataFrame(y_val_segmentid)

In [ ]:
df_y_val_segmentid

In [ ]:
#df_y_val_uttid.columns = ['segmentid']    


In [ ]:
#df_y_val_uttid

In [ ]:
yy = pd.concat([df_y_val_segmentid,df_post],axis=1)

In [ ]:
yy

In [ ]:
#yy.to_csv('/home/satishk/ivectors_csv/test_dev_30sec.csv',index=False)

In [ ]:
#yy['segmentid'] = yy['segmentid'].astype(str) + '.sph'

In [ ]:
#yy.to_csv('/home/satishk/ivectors_csv/test_dev_01oct.csv',mode = 'a',index=False,header=False)

In [ ]:
yy.to_csv('/home/satishk/ivectors_csv/test_dev_16oct_Bharat_160k.csv',index=False)

In [ ]:
test_dev_scr = pd.read_csv('/home/satishk/ivectors_csv/test_dev_16oct_Bharat_160k.csv')


In [ ]:
test_dev_scr

In [ ]:
test_dev_scr.to_csv('/home/satishk/lre17_eval_framework/system_output/test_dev_16oct.scr', header=True, index=False, sep='\t')
